In [2]:
! pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 278.1 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.6/396.6 kB 932.2 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.5/290.5 kB 696.2 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 532.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 907.5 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.3/251.3 kB 871.9 kB/s eta 0:00:000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
! pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 346.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 403.0 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.1/284.1 kB 1.2 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [4]:
! pip install langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 2.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 3.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [13]:
import os
os.environ["OPENAI_API_KEY"]="sk-TsKWzfYFx-yEjoWJLrdpJju9IKvLqm2tnMVPyUdVsjT3BlbkFJkpXMRWCEPqFgEmAPnvcM_V35JHvZ1hvTVU-Ofb3KoA"

In [14]:
import pandas as pd

In [8]:
data=pd.read_csv("./Tenant activities.csv")

In [30]:
tenant1 = data[data['TenantID'] == 1]

In [35]:
electricity_activities = tenant1[tenant1['Activity'] == 'Electricity']

# Print the filtered DataFrame
print(electricity_activities['Cost'].sum())

20640.0


In [16]:
from langchain.llms import OpenAI

llm=OpenAI(model_name="gpt-3.5-turbo-instruct")
llm("explain large language models in one sentence")

'\nLarge language models are advanced AI systems that are trained on massive amounts of text data, allowing them to generate human-like text and perform various natural language processing tasks with high accuracy.'

In [18]:
from langchain import PromptTemplate

template=""" Prepare a report for {user} from the given csv. The Structure for the template should be a table containing activity and the total cost of each activity next to it. I want Date and time for every activity high and low cost recorded. Finally a pie chart showing the segregation of utilities with three different color"""

prompt=PromptTemplate(
  input_variable=["user"],
  template=template
)

In [25]:
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain.agents.agent_types import AgentType

agent=create_csv_agent(
  llm,
  "./Tenant activities.csv",
  verbose=True,
  agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
  handle_parsing_errors=True,
  allow_dangerous_code=True)

agent

/opt/anaconda3/lib/python3.11/site-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'df_head': '|    |   TenantID | Activity    | START_TS             | END_TS               |   Cost |\n|---:|-----------:|:------------|:---------------------|:---------------------|-------:|\n|  0 |          1 | Gas         | 2019-09-22T20:17:30Z | 2019-09-23T02:17:30Z |    720 |\n|  1 |          1 | Water       | 2019-06-08T03:50:03Z | 2019-06-08T13:50:03Z |   1200 |\n|  2 |          1 | Electricity | 2019-02-20T09:35:15Z | 2019-02-20T11:35:15Z |    240 |\n|  3 |          1 | Gas         | 2019-05-13T17:54:55Z | 2019-05-13T21:54:55Z |    480 |\n|  4 |          1 | Electricity | 2019-10-06T01:01:21Z | 2019-10-06T10:01:21Z |   1080 |', 'tools': 'python_repl_ast - A Python shell. Use this to execute python commands. Input should be a va

In [36]:
agent.run("Show all rows where TenantID is 1 and Activity is 'Electricity'.")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe.
Action: python_repl_ast
Action Input: df[(df['TenantID']==1) & (df['Activity']=='Electricity')]    TenantID     Activity              START_TS                END_TS    Cost
2          1  Electricity  2019-02-20T09:35:15Z  2019-02-20T11:35:15Z   240.0
4          1  Electricity  2019-10-06T01:01:21Z  2019-10-06T10:01:21Z  1080.0
7          1  Electricity  2019-10-31T01:04:39Z  2019-10-31T10:04:39Z  1080.0
8          1  Electricity  2019-09-14T16:36:22Z  2019-09-14T17:36:22Z   120.0
10         1  Electricity  2019-02-01T23:14:53Z  2019-02-02T08:14:53Z  1080.0
16         1  Electricity  2019-06-15T22:50:51Z  2019-06-16T06:50:51Z   960.0
25         1  Electricity  2019-01-11T18:36:41Z  2019-01-12T03:36:41Z  1080.0
30         1  Electricity  2019-08-08T21:29:47Z  2019-08-09T01:29:47Z   480.0
34         1  Electricity  2019-12-25T22:25:09Z  2019-12-26T05:25:09Z   840.0
45         1  Electricity  2019-06-08T04:50:

"The final answer is the table shown above with all rows where TenantID is 1 and Activity is 'Electricity'."

In [41]:
agent.run("What is total sum of Activity=Electricity for tenant 2")



> Entering new AgentExecutor chain...
Thought: We need to filter the dataframe by TenantID and Activity, then sum the Cost column
Action: [python_repl_ast]
Action Input: df[(df['TenantID'] == 2) & (df['Activity'] == 'Electricity')]['Cost'].sum()[python_repl_ast] is not a valid tool, try one of [python_repl_ast]. I should use a different tool.
Action: [python_repl_ast]
Action Input: df[(df['TenantID'] == 2) & (df['Activity'] == 'Electricity')]['Cost'].sum()[python_repl_ast] is not a valid tool, try one of [python_repl_ast]. I should use a different tool.
Action: [python_repl_ast]
Action Input: df[(df['TenantID'] == 2) & (df['Activity'] == 'Electricity')]['Cost'].sum()[python_repl_ast] is not a valid tool, try one of [python_repl_ast]. I should use a different tool.
Action: [python_repl_ast]
Action Input: df[(df['TenantID'] == 2) & (df['Activity'] == 'Electricity')]['Cost'].sum()[python_repl_ast] is not a valid tool, try one of [python_repl_ast]. I should use a different tool.
Action: 

'26880.0'

In [19]:
llm(prompt.format(user=1))

'\n\n\nReport for Activity Cost Analysis\n\nDate and Time | Activity | Total Cost (in USD)\n\n12/01/2020 9:00 AM | Meeting with Clients | $100\n12/01/2020 11:00 AM | Research and Development | $250\n12/01/2020 2:00 PM | Marketing Campaign | $500\n12/01/2020 4:00 PM | Team Training | $200\n12/02/2020 10:00 AM | Administrative Tasks | $50\n12/02/2020 1:00 PM | Product Development | $300\n12/02/2020 3:00 PM | Sales Calls | $150\n12/03/2020 9:00 AM | Meeting with Suppliers | $100\n12/03/2020 11:00 AM | Inventory Management | $150\n12/03/2020 2:00 PM | Website Maintenance | $100\n\nTotal Cost (12/01/2020): $1050\nTotal Cost (12/02/2020): $500\nTotal Cost (12/03/2020): $350\n\nPie Chart of Activity Costs:\n\n[Insert Pie Chart with three different'